In [42]:
import re
from os import listdir
from os.path import isfile, join
import csv
import os
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
train_dir = '/home/hoang/Downloads/train/'
files = [f for f in listdir(train_dir) if isfile(join(train_dir, f))]

class CSVProcessor:
    def __init__(self, fdir, fname):
        self.fdir = fdir
        self.fname = fname
        
    def process(self):
        line = 0
        df = pd.read_csv(filepath_or_buffer=os.path.join(self.fdir, self.fname))
        return df
    
proc = CSVProcessor('data/', 'trainLabels.csv')
df = proc.process()

train, test = train_test_split(df, test_size=0.8, random_state=42) # train on 40% given data

In [43]:
train_hashes = [x for x in train['Id']]
train_classes = [x for x in train['Class']]

In [57]:
class GetDf:
    def __init__(self, data):
        self.hashes = [x for x in data['Id']]
        self.classes = [x for x in data['Class']]
        
    def get_df(self):
        df = pd.DataFrame(columns=['hashes', 'asm', 'bytes', 'class'])
        df['hashes'] = self.hashes
        df['class'] = self.classes
        df['asm'] = [os.stat(join(train_dir, f + '.asm')).st_size for f in self.hashes]
        df['bytes'] = [os.stat(join(train_dir, f + '.bytes')).st_size for f in self.hashes]
        
#         scaler = StandardScaler()
#         scaler.fit(df[['asm', 'bytes']])
#         df[['asm', 'bytes']] = scaler.transform(df[['asm', 'bytes']])
        
        return df

In [69]:
from sklearn.ensemble import RandomForestClassifier

rdc = RandomForestClassifier(n_estimators=400, max_depth=int(np.sqrt(2173)))

In [70]:
train_df = GetDf(train).get_df()

In [71]:
train_df.shape

(2173, 4)

In [72]:
rdc.fit(train_df[['asm', 'bytes']], train_df[['class']])

/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=46, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [73]:
from sklearn.metrics import accuracy_score

test_df = GetDf(test).get_df()

accuracy_score(rdc.predict(test_df[['asm', 'bytes']]), test_df['class'])

0.9053479010925819

In [80]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7).fit(train_df[['asm', 'bytes']], train_df[['class']])

/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
knn.score(test_df[['asm', 'bytes']], test_df[['class']])

0.8615296147211041

In [82]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=200)

In [83]:
ada.fit(train_df[['asm', 'bytes']], train_df['class'])

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [85]:
ada.score(test_df[['asm', 'bytes']], test_df['class'])

0.9048878665899942

In [87]:
dtc = DecisionTreeClassifier(max_depth=10)
dtc.fit(train_df[['asm', 'bytes']], train_df['class'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [88]:
dtc.score(test_df[['asm', 'bytes']], test_df['class'])

0.8726854514088557